### Setup (only required for the first run on the Spark cluster)

In [1]:
!pip install pandas

    100% |████████████████████████████████| 10.1MB 3.9MB/s eta 0:00:01
pyspark 2.3.2 requires py4j==0.10.7, which is not installed.
You are using pip version 10.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
ROOT_PATH = 'gs://news_public_datasets4/adressa'
#TODO: Upload this file (generated by the ACR module training) to GCS before calling spark script
!gsutil cp {ROOT_PATH}/data_transformed/adressa_articles.csv .

Copying gs://news_public_datasets4/adressa/data_transformed/adressa_articles.csv...
- [1 files][ 60.3 MiB/ 60.3 MiB]                                                
Operation completed over 1 objects/60.3 MiB.                                     


### Loading dependencies

In [3]:
import os
import json
import pandas as pd
import pickle
import datetime
import hashlib
import math
import matplotlib
%matplotlib inline

In [4]:
import pyspark
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.sql.functions import pandas_udf
from pyspark.sql.functions import PandasUDFType

## Loading articles pre-processed

In [5]:
articles_original_df = pd.read_csv('adressa_articles.csv')

In [6]:
articles_original_df.columns

Index(['adressa-access', 'author_1st', 'category0', 'category1', 'category2',
       'concepts', 'created_at_ts', 'entities', 'id', 'keywords', 'locations',
       'persons', 'publishtime', 'site', 'text_highlights', 'url',
       'id_encoded', 'category0_encoded', 'category1_encoded',
       'author_encoded', 'keywords_encoded', 'concepts_encoded',
       'entities_encoded', 'locations_encoded', 'persons_encoded'],
      dtype='object')

In [7]:
articles_original_df['url'].nunique()

72932

In [8]:
valid_articles_urls_to_ids_dict = dict(articles_original_df[['url','id_encoded']].apply(lambda x: (x['url'], x['id_encoded']), axis=1).values)
len(valid_articles_urls_to_ids_dict)

72932

### Loading user interactions

In [9]:
#INTERACTIONS_PATH = 'gs://news_public_datasets4/adressa/one_week/*'
#INTERACTIONS_PATH = 'three_month/20170101'

DAYS_TO_LOAD_INTERACTIONS=17
interaction_json_files = [os.path.join(ROOT_PATH, 'three_month/201701{:02d}'.format(day)) for day in range(1, DAYS_TO_LOAD_INTERACTIONS)]
print('Loading interaction files: {}'.format(interaction_json_files))

interactions_df = spark.read \
  .option("mode", "PERMISSIVE") \
  .json(interaction_json_files)

Loading interaction files: ['gs://news_public_datasets4/adressa/three_month/20170101', 'gs://news_public_datasets4/adressa/three_month/20170102', 'gs://news_public_datasets4/adressa/three_month/20170103', 'gs://news_public_datasets4/adressa/three_month/20170104', 'gs://news_public_datasets4/adressa/three_month/20170105', 'gs://news_public_datasets4/adressa/three_month/20170106', 'gs://news_public_datasets4/adressa/three_month/20170107', 'gs://news_public_datasets4/adressa/three_month/20170108', 'gs://news_public_datasets4/adressa/three_month/20170109', 'gs://news_public_datasets4/adressa/three_month/20170110', 'gs://news_public_datasets4/adressa/three_month/20170111', 'gs://news_public_datasets4/adressa/three_month/20170112', 'gs://news_public_datasets4/adressa/three_month/20170113', 'gs://news_public_datasets4/adressa/three_month/20170114', 'gs://news_public_datasets4/adressa/three_month/20170115', 'gs://news_public_datasets4/adressa/three_month/20170116']


In [10]:
interactions_df.printSchema()

root
 |-- activeTime: long (nullable = true)
 |-- author: string (nullable = true)
 |-- canonicalUrl: string (nullable = true)
 |-- category1: string (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- deviceType: string (nullable = true)
 |-- eventId: long (nullable = true)
 |-- id: string (nullable = true)
 |-- keywords: string (nullable = true)
 |-- os: string (nullable = true)
 |-- profile: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- groups: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- count: long (nullable = true)
 |    |    |    |    |-- group: string (nullable = true)
 |    |    |    |    |-- weight: double (nullable = true)
 |    |    |-- item: string (nullable = true)
 |-- publishtime: string (nullable = true)
 |-- query: string (nullable = true)
 |-- referrerHostClass: string (nullable = true)
 |-- referrerQuery: string (nullab

In [11]:
interactions_df.count()

21134428

In [12]:
#Retrives article id from its cannonical URL (because sometimes article ids in interactions do no match with articles tables, but cannonical URL do)
def get_article_id_encoded_from_url(canonical_url):
    if canonical_url in valid_articles_urls_to_ids_dict:
        return valid_articles_urls_to_ids_dict[canonical_url]    
    return None

get_article_id_encoded_from_url_udf = F.udf(get_article_id_encoded_from_url, pyspark.sql.types.IntegerType())

In [13]:
#Filtering only interactions whose url/id is available in the articles table
interactions_article_id_encoded_df = interactions_df.withColumn('article_id', get_article_id_encoded_from_url_udf(interactions_df['canonicalUrl']))
interactions_filtered_df = interactions_article_id_encoded_df.filter(interactions_article_id_encoded_df['article_id'].isNull() == False).cache()

In [14]:
#Valid interactions
interactions_filtered_df.count()

5483902

In [15]:
#Distinct items count
interactions_filtered_df.select('article_id').distinct().count()

26873

In [16]:
first_timestamp_ts = interactions_filtered_df.select('time').agg(F.min('time')).collect()[0][0] * 1000
first_timestamp_ts

1483225202000

### Analyzing elapsed time since publishing

In [17]:
#interactions_filtered_df.filter(interactions_filtered_df['time'].isNull()).count()
#0

In [18]:
def get_timestamp_from_date_str(value):
    if value is not None:
        return int(datetime.datetime.strptime(value, '%Y-%m-%dT%H:%M:%S.%fZ').timestamp())
    return None

get_timestamp_from_date_str_udf = F.udf(get_timestamp_from_date_str, pyspark.sql.types.IntegerType())

In [19]:
interactions_filtered_with_publish_ts_df = interactions_filtered_df.withColumn('publish_ts', get_timestamp_from_date_str_udf(interactions_filtered_df['publishtime']))
interactions_filtered_with_publish_ts_df = interactions_filtered_with_publish_ts_df.withColumn('elapsed_min_since_published', ((F.col('time') - F.col('publish_ts')) / 60).cast(pyspark.sql.types.IntegerType()))

In [20]:
#interactions_filtered_with_publish_ts_df.select('publishtime','publish_ts', 'time', 'elapsed_min_since_published').show(100)

In [21]:
%%time
interactions_filtered_with_publish_ts_df.approxQuantile("elapsed_min_since_published", [0.10, 0.25, 0.50, 0.75, 0.90], 0.01)
#[49.0, 108.0, 334.0, 1020.0, 4611.0]

CPU times: user 40 ms, sys: 12 ms, total: 52 ms
Wall time: 26.5 s


[49.0, 106.0, 314.0, 984.0, 4145.0]

In [22]:
elapsed_min_since_published_df = interactions_filtered_with_publish_ts_df.select('elapsed_min_since_published').toPandas()
print(len(elapsed_min_since_published_df[pd.isnull(elapsed_min_since_published_df['elapsed_min_since_published'])]))
elapsed_min_since_published_df.describe()

45551


,elapsed_min_since_published
count,5.438351e+06
mean,7.815292e+04
std,5.645131e+05
min,-3.151590e+05
25%,9.800000e+01
50%,2.940000e+02
75%,9.400000e+02
max,1.144721e+07


In [23]:

'''
elapsed_min_since_published
count	2.600818e+06
mean	6.438622e+04
std	5.051825e+05
min	-3.151590e+05
25%	9.400000e+01
50%	2.580000e+02
75%	8.370000e+02
max	8.608278e+06
'''

'\nelapsed_min_since_published\ncount\t2.600818e+06\nmean\t6.438622e+04\nstd\t5.051825e+05\nmin\t-3.151590e+05\n25%\t9.400000e+01\n50%\t2.580000e+02\n75%\t8.370000e+02\nmax\t8.608278e+06\n'

### Analyzing clicks by article distribution

In [24]:
#clicks_by_article_count_df = interactions_filtered_df.groupBy('article_id').count()
#clicks_by_article_count_df.approxQuantile("count", [0.01, 0.10, 0.25, 0.50, 0.75, 0.90, 0.99], 0.01)
#[1.0, 1.0, 1.0, 1.0, 2.0, 6.0, 33581.0]

### Processing categorical features

In [25]:
def get_categ_features_counts_dataframe(interactions_spark_df,column_name):
    df_pandas = interactions_spark_df.groupBy(column_name).count().toPandas().sort_values('count', ascending=False)
    return df_pandas

In [26]:
PAD_TOKEN = '<PAD>'
UNFREQ_TOKEN = '<UNF>'

def get_encoder_for_values(values):
    encoder_values = [PAD_TOKEN, UNFREQ_TOKEN] + values
    encoder_ids = list(range(len(encoder_values)))
    encoder_dict = dict(zip(encoder_values, encoder_ids))
    return encoder_dict

def get_categ_features_encoder_dict(counts_df, min_freq=100):
    freq_values = counts_df[counts_df['count'] >= 100][counts_df.columns[0]].values.tolist()
    encoder_dict = get_encoder_for_values(freq_values)
    return encoder_dict

def encode_cat_feature(value, encoder_dict):
    if value in encoder_dict:
        return encoder_dict[value]
    else:
        return encoder_dict[UNFREQ_TOKEN]

In [27]:
countries_df = get_categ_features_counts_dataframe(interactions_filtered_df, 'country')
len(countries_df)

178

In [28]:
countries_encoder_dict = get_categ_features_encoder_dict(countries_df)
len(countries_encoder_dict)

70

In [29]:
cities_df = get_categ_features_counts_dataframe(interactions_filtered_df, 'city')
len(cities_df)

7002

In [30]:
cities_encoder_dict = get_categ_features_encoder_dict(cities_df)
len(cities_encoder_dict)

1022

In [31]:
regions_df = get_categ_features_counts_dataframe(interactions_filtered_df, 'region')
len(regions_df)

1353

In [32]:
regions_encoder_dict = get_categ_features_encoder_dict(regions_df)
len(regions_encoder_dict)

237

In [33]:
devices_df = get_categ_features_counts_dataframe(interactions_filtered_df, 'deviceType')
print(len(devices_df))
devices_df

3


,deviceType,count
0,Mobile,2667039
2,Desktop,1716787
1,Tablet,1100076


In [34]:
devices_encoder_dict = get_categ_features_encoder_dict(devices_df)
len(devices_encoder_dict)

5

In [35]:
os_df = get_categ_features_counts_dataframe(interactions_filtered_df, 'os')
print(len(os_df))
os_df

11


,os,count
0,iPhone OS,2350549
10,Windows,1460956
8,Android,1395958
3,Macintosh,235810
2,Windows Phone OS,18670
5,Linux,15296
7,Unknown,6309
4,BlackBerry,288
1,Symbian OS,42
9,BSD,14


In [36]:
os_encoder_dict = get_categ_features_encoder_dict(os_df)
len(os_encoder_dict)

10

In [37]:
referrer_class_df = get_categ_features_counts_dataframe(interactions_filtered_df, 'referrerHostClass')
print(len(referrer_class_df))
referrer_class_df

5


,referrerHostClass,count
2,internal,3259105
4,search,776911
0,direct,772549
3,social,421781
1,other,253556


In [38]:
referrer_class_encoder_dict = get_categ_features_encoder_dict(referrer_class_df)
len(referrer_class_encoder_dict)

7

In [39]:
encoders_dict = {
    'city': cities_encoder_dict,
    'region': regions_encoder_dict,
    'country': countries_encoder_dict,
    'os': os_encoder_dict,
    'device': devices_encoder_dict,
    'referrer_class': referrer_class_encoder_dict
}

### Processing numeric features

In [40]:
%%time
active_time_quantiles = interactions_filtered_df.approxQuantile("activeTime", [0.10, 0.25, 0.50, 0.75, 0.90], 0.01)
print(active_time_quantiles)

[8.0, 21.0, 47.0, 90.0, 158.0]
CPU times: user 36 ms, sys: 16 ms, total: 52 ms
Wall time: 38.2 s


In [41]:
active_time_stats_df = interactions_filtered_df.describe('activeTime').toPandas()
active_time_stats_df

,summary,activeTime
0,count,2320994
1,mean,65.00062602488417
2,stddev,69.37057327780722
3,min,1
4,max,899


In [42]:
active_time_mean = float(active_time_stats_df[active_time_stats_df['summary'] == 'mean']['activeTime'].values[0])
active_time_stddev = float(active_time_stats_df[active_time_stats_df['summary'] == 'stddev']['activeTime'].values[0])

### Splitting sessions

In [43]:
'''
schema = T.StructType([
    T.StructField("userId", T.StringType()),
    T.StructField("min_ts", T.IntegerType())
])

@pandas_udf(schema, functionType=PandasUDFType.GROUPED_MAP)
def split_sessions(df):

    result_df = df[['userId']]
    result_df['min_ts'] = df['time'].min()
    
    return result

%%time
tmp = interactions_filtered_df.groupBy('userId').apply(split_sessions)
tmp.show(100)
'''

'\nschema = T.StructType([\n    T.StructField("userId", T.StringType()),\n    T.StructField("min_ts", T.IntegerType())\n])\n\n@pandas_udf(schema, functionType=PandasUDFType.GROUPED_MAP)\ndef split_sessions(df):\n\n    result_df = df[[\'userId\']]\n    result_df[\'min_ts\'] = df[\'time\'].min()\n    \n    return result\n\n%%time\ntmp = interactions_filtered_df.groupBy(\'userId\').apply(split_sessions)\ntmp.show(100)\n'

In [44]:
def hash_str_to_int(encoded_bytes_text, digits):
    return int(str(int(hashlib.md5(encoded_bytes_text).hexdigest()[:8], 16))[:digits])      

In [45]:
MAX_SESSION_IDLE_TIME_MS = 30 * 60 * 1000    #30 min

def close_session(session):
    size = len(session)
    
    #Creating and artificial session id based on the first click timestamp and a hash of user id
    first_click = session[0]
    session_id = (int(first_click['timestamp']) * 100) + hash_str_to_int(first_click['user_id'].encode(), 3)
    session_hour = int((first_click['timestamp'] - first_timestamp_ts) / (1000 * 60 * 60)) #Converting timestamp to hours since first timestamp
    
    #Converting to Spark DataFrame Rows, to convert RDD back to DataFrame
    clicks = list([T.Row(**click) for click in session])
    session_dict = {'session_id': session_id,
                    'session_hour': session_hour,
                    'session_size': size,
                    'session_start': first_click['timestamp'],
                    'user_id': first_click['user_id'],
                    'clicks': clicks 
                   }
    session_row = T.Row(**session_dict)
    
    return session_row
        
def transform_interaction(interaction):        
    return {
            'article_id': interaction['article_id'],
            'url': interaction['canonicalUrl'],
            'user_id': interaction['userId'],
            'timestamp': interaction['time'] * 1000, #converting to timestamp
            'active_time_secs': interaction['activeTime'],
            'country': encode_cat_feature(interaction['country'], encoders_dict['country']),
            'region': encode_cat_feature(interaction['region'], encoders_dict['region']),
            'city': encode_cat_feature(interaction['city'], encoders_dict['city']),
            'os': encode_cat_feature(interaction['os'], encoders_dict['os']),
            'device': encode_cat_feature(interaction['deviceType'], encoders_dict['device']),
            'referrer_class': encode_cat_feature(interaction['referrerHostClass'], encoders_dict['referrer_class']),
           }

def split_sessions(group):
    user, interactions = group
    #Ensuring items are sorted by time
    interactions_sorted_by_time = sorted(interactions, key=lambda x: x['time'])
    #Transforming interactions
    interactions_transformed = list(map(transform_interaction, interactions_sorted_by_time))

    
    sessions = []
    session = []        
    first_timestamp = interactions_transformed[0]['timestamp']
    last_timestamp = first_timestamp    
    for interaction in interactions_transformed:
        
        delta_ms = (interaction['timestamp'] - last_timestamp)
        interaction['_elapsed_ms_since_last_click'] = delta_ms 

        if delta_ms <= MAX_SESSION_IDLE_TIME_MS:    
            #Ignoring repeated items in session
            if len(list(filter(lambda x: x['article_id'] == interaction['article_id'], session))) == 0:        
                session.append(interaction)            
        else:
            #If session have at least 2 clicks (minimum for next click predicition)
            if len(session) >= 2:
                session_row = close_session(session)
                sessions.append(session_row)                
            session = [interaction]

        last_timestamp = interaction['timestamp']
            
    if len(session) >= 2:
        session_row = close_session(session)
        sessions.append(session_row)
        
    #if len(sessions) > 1:
    #    raise Exception('USER with more than one session: {}'.format(user))
    
    return list(zip(map(lambda x: x['session_id'], sessions), 
                    sessions))

In [46]:
'''
#To debug
%%time
sessions_rdd = interactions_filtered_df.limit(1000).rdd.map(lambda x: (x['userId'], x)).groupByKey() \
                    .collect()

for row in sessions_rdd:
    print(split_sessions(row))
    print()
'''

"\n#To debug\n%%time\nsessions_rdd = interactions_filtered_df.limit(1000).rdd.map(lambda x: (x['userId'], x)).groupByKey()                     .collect()\n\nfor row in sessions_rdd:\n    print(split_sessions(row))\n    print()\n"

In [47]:
%%time
sessions_rdd = interactions_filtered_df.rdd.map(lambda x: (x['userId'], x)).groupByKey() \
                            .flatMap(split_sessions) \
                            .sortByKey() \
                            .map(lambda x: x[1])

CPU times: user 304 ms, sys: 84 ms, total: 388 ms
Wall time: 27min 59s


#### Exporting sessions to JSON lines

In [48]:
sessions_sdf = sessions_rdd.toDF()

In [49]:
%%time
sessions_sdf.write.partitionBy("session_hour").json(os.path.join(ROOT_PATH,"data_transformed/sessions_processed_by_spark/"))

CPU times: user 56 ms, sys: 12 ms, total: 68 ms
Wall time: 3min 49s


In [50]:
sessions_sdf.count()

982210

In [51]:
def serialize(filename, obj):
    with open(filename, 'wb') as handle:
        pickle.dump(obj, handle)#, protocol=pickle.HIGHEST_PROTOCOL)

In [52]:
NAR_ENCODERS_PATH  = 'nar_encoders_adressa.pickle'
serialize(NAR_ENCODERS_PATH, encoders_dict)

In [53]:
!gsutil cp {NAR_ENCODERS_PATH} {ROOT_PATH}/data_transformed/pickles/

Copying file://nar_encoders_adressa.pickle [Content-Type=application/octet-stream]...
/ [1 files][ 26.5 KiB/ 26.5 KiB]                                                
Operation completed over 1 objects/26.5 KiB.                                     
